In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R1_1/models/B5_R1_AN_relu_1FC_fold1_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(7392, 22)
(7392, 22)
Normal:  (4004, 22)
Abnormal:  (3388, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,0,1,42,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,231.0,134.0,544.0,380.0,0.229956,0.149621,0.541516,0.422222,AB02 P1 C042.JPG
1,1,6,42,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,229.0,98.0,645.0,550.0,0.228564,0.109217,0.643761,0.612121,AB04 P1 C042.JPG
2,2,10,42,P1,P1,Abnormal,AB06,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,267.0,45.0,564.0,525.0,0.263207,0.050631,0.555357,0.583838,AB06 P1 C042.JPG
3,3,20,22,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,200.0,141.0,589.0,501.0,0.201802,0.156841,0.592793,0.557769,AB01 P1 C022.JPG
4,4,29,22,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,155.0,67.0,506.0,639.0,0.166464,0.074874,0.543860,0.711111,AB02 P1 C022.JPG


In [6]:
# dataframe.info()

In [7]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 7392 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [8]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [9]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.06255286,  0.18608114, -0.11897527, ..., -0.03046142,
         0.40195188, -0.00276822],
       [-0.15452538,  0.15598406, -0.1576266 , ..., -0.10438192,
         0.38413697, -0.05860931],
       [-0.08839631,  0.05133146, -0.16588823, ...,  0.04720832,
         0.30860138,  0.00831166],
       ...,
       [-0.08838294,  0.26937771, -0.14912085, ..., -0.09899292,
         0.33413297,  0.01399111],
       [-0.00896431,  0.12289834, -0.08917272, ..., -0.08282755,
         0.2988587 , -0.0021148 ],
       [-0.08956655,  0.16025807, -0.15371564, ..., -0.05277725,
         0.40388757, -0.02708807]])

In [10]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.062553,0.186081,-0.118975,-0.091163,0.885550,-0.186141,-0.080746,0.458904,-0.039819,-0.170707,...,-0.110820,0.103643,0.213937,0.305316,-0.218449,0.084175,0.680296,-0.030461,0.401952,-0.002768
1,-0.154525,0.155984,-0.157627,-0.056073,0.838088,-0.122291,-0.185961,0.386215,0.026218,-0.172814,...,-0.099115,-0.065673,0.015662,-0.040467,-0.122136,-0.034675,0.273085,-0.104382,0.384137,-0.058609
2,-0.088396,0.051331,-0.165888,0.055930,0.538104,-0.119353,-0.091007,0.694948,-0.159082,-0.206403,...,-0.064128,-0.137454,0.286648,-0.176649,-0.213272,-0.015313,0.774554,0.047208,0.308601,0.008312
3,-0.122556,0.200062,-0.070700,-0.032537,0.889488,-0.155962,-0.136352,0.341719,-0.064425,-0.213977,...,-0.163622,-0.089903,0.025749,0.123896,-0.170233,0.109580,0.050536,-0.107618,0.347555,-0.029911
4,-0.138276,0.030288,-0.056607,0.075901,0.698406,-0.113334,-0.097215,0.164659,-0.073760,-0.224023,...,-0.179709,-0.023883,0.511097,-0.122256,-0.185832,0.157855,0.385003,0.031002,0.705700,0.053334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,-0.137797,0.165149,-0.056478,0.162836,0.819958,-0.172885,-0.082179,0.178332,-0.006301,-0.205239,...,-0.218890,0.013376,0.188125,-0.094243,-0.066810,0.094312,0.160537,-0.093524,0.328881,0.010056
7388,-0.051143,0.137159,-0.127429,-0.021891,0.915363,-0.188141,-0.087184,0.244964,-0.056451,-0.204275,...,-0.099688,0.049587,0.248408,0.197955,-0.195977,-0.076908,0.591621,-0.118047,0.393148,-0.037323
7389,-0.088383,0.269378,-0.149121,0.119399,0.746629,-0.168337,-0.140151,0.229694,-0.077905,-0.104289,...,-0.180092,-0.086785,0.232821,0.071634,-0.187943,0.312497,0.293288,-0.098993,0.334133,0.013991
7390,-0.008964,0.122898,-0.089173,0.174791,1.107949,-0.188351,-0.136421,0.225617,0.058631,-0.179442,...,-0.121641,0.037290,0.313175,0.194776,-0.152722,0.262708,0.520532,-0.082828,0.298859,-0.002115


In [11]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [12]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.062553,0.186081,-0.118975,-0.091163,0.885550,-0.186141,-0.080746,0.458904,-0.039819,-0.170707,...,-0.218449,0.084175,0.680296,-0.030461,0.401952,-0.002768,Abnormal,P1,FP-A,AB02
1,-0.154525,0.155984,-0.157627,-0.056073,0.838088,-0.122291,-0.185961,0.386215,0.026218,-0.172814,...,-0.122136,-0.034675,0.273085,-0.104382,0.384137,-0.058609,Abnormal,P1,FP-A,AB04
2,-0.088396,0.051331,-0.165888,0.055930,0.538104,-0.119353,-0.091007,0.694948,-0.159082,-0.206403,...,-0.213272,-0.015313,0.774554,0.047208,0.308601,0.008312,Abnormal,P1,FP-A,AB06
3,-0.122556,0.200062,-0.070700,-0.032537,0.889488,-0.155962,-0.136352,0.341719,-0.064425,-0.213977,...,-0.170233,0.109580,0.050536,-0.107618,0.347555,-0.029911,Abnormal,P1,FP-A,AB01
4,-0.138276,0.030288,-0.056607,0.075901,0.698406,-0.113334,-0.097215,0.164659,-0.073760,-0.224023,...,-0.185832,0.157855,0.385003,0.031002,0.705700,0.053334,Abnormal,P1,FP-A,AB02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,-0.137797,0.165149,-0.056478,0.162836,0.819958,-0.172885,-0.082179,0.178332,-0.006301,-0.205239,...,-0.066810,0.094312,0.160537,-0.093524,0.328881,0.010056,Normal,P8,FP-E,Normal
7388,-0.051143,0.137159,-0.127429,-0.021891,0.915363,-0.188141,-0.087184,0.244964,-0.056451,-0.204275,...,-0.195977,-0.076908,0.591621,-0.118047,0.393148,-0.037323,Normal,P8,FP-E,Normal
7389,-0.088383,0.269378,-0.149121,0.119399,0.746629,-0.168337,-0.140151,0.229694,-0.077905,-0.104289,...,-0.187943,0.312497,0.293288,-0.098993,0.334133,0.013991,Normal,P8,FP-E,Normal
7390,-0.008964,0.122898,-0.089173,0.174791,1.107949,-0.188351,-0.136421,0.225617,0.058631,-0.179442,...,-0.152722,0.262708,0.520532,-0.082828,0.298859,-0.002115,Normal,P8,FP-E,Normal


In [14]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold1_3.csv')